### AMIE rules before and after KG enrichment using RotatE

In [1]:
#applying AnyBURL rules on openbiolink test set took 22464199ms
#applying RotAMIE rules on openbiolink test set took22432498ms
#(#1167600 rule application steps in DRKG using RotAMIE)
#AnyBURL rule application took #12400 steps on OPENBIOLINK for a total runtime of 68400 seconds. Estimated time on full dataset is 68400*(877846/12400)

In [3]:
# Some rules may be the same but use different variables. We make sure to identify those as the same, using the same variables! 

def maybe_replace_variable(rule):
    all_variables = ["?a", "?b", "?g", "?h"]
    for var1 in all_variables[::-1]:
        if var1 in rule:
            for var2 in all_variables:
                if not var2 in rule:
                    rule = rule.replace(var1, var2)
                    break
                if var2 >= var1:
                    break
    return rule

In [4]:
def compare_rules_amie(rules_before, rules_after):
    with open(rules_before) as file:
        Rules_1 = file.read().strip().split('\n')
    with open(rules_after) as file:
        Rules_2 = file.read().strip().split('\n')
    rules_1 = list(filter(lambda x: x.startswith("?"), Rules_1))
    rules_2 = list(filter(lambda x: x.startswith("?"), Rules_2))
    rules_1 = list(map(maybe_replace_variable, rules_1))
    rules_2 = list(map(maybe_replace_variable, rules_2))
    Rules_1 = set(map(lambda x: x.split('\t')[0], rules_1))
    Rules_2 = set(map(lambda x: x.split('\t')[0], rules_2))
    
    new_rules = Rules_2-Rules_1
    dropped_rules = Rules_1-Rules_2
    
    same_rules = Rules_1.intersection(Rules_2)
    
    new_rules = sorted(filter(lambda x: x.split('\t')[0] in new_rules, rules_2), key=lambda t: float(t.split('\t')[3]), reverse=True)
    dropped_rules = sorted(filter(lambda x: x.split('\t')[0] in dropped_rules, rules_1), key=lambda t: float(t.split('\t')[3]), reverse=True)
    same_rules_1 = sorted(filter(lambda x: x.split('\t')[0] in same_rules, rules_1), key=lambda t: float(t.split('\t')[3]), reverse=True)
    same_rules_2 = sorted(filter(lambda x: x.split('\t')[0] in same_rules, rules_2), key=lambda t: float(t.split('\t')[3]), reverse=True)
    rules_1 = sorted(rules_1, key=lambda t: float(t.split('\t')[3]), reverse=True)
    rules_2 = sorted(rules_2, key=lambda t: float(t.split('\t')[3]), reverse=True)
    
    print(f'There were {len(Rules_1)} rules before, there are {len(Rules_2)} after enrichment')
    print()
    print(f'There are {len(same_rules)} same rules, {len(new_rules)} new rules, {len(dropped_rules)} dropped rules')
    
    return same_rules_1, same_rules_2, rules_1, rules_2, new_rules, dropped_rules

## WN18RR

In [5]:
same_rules_amie_wn18rr_1, same_rules_amie_wn18rr_2, rules_amie_before_wn18rr, rules_amie_after_wn18rr, \
new_rules_amie_wn18rr, dropped_rules_amie_wn18rr = compare_rules_amie('AMIE/rules_wn18rr/rules.txt', 'RotAMIE/rules_wn18rr/rules_top500.txt')

There were 34 rules before, there are 28 after enrichment

There are 26 same rules, 2 new rules, 8 dropped rules


In [6]:
same_rules_amie_wn18rr_1[:5]

['?b  _verb_group  ?a   => ?a  _verb_group  ?b\t0.931458699\t0.931458699\t0.980573543\t1060\t1138\t1081\t?b',
 '?b  _derivationally_related_form  ?a   => ?a  _derivationally_related_form  ?b\t0.932222783\t0.932222783\t0.95163008\t27701\t29715\t29109\t?b',
 '?b  _also_see  ?a   => ?a  _also_see  ?b\t0.637413395\t0.637413395\t0.883671291\t828\t1299\t937\t?b',
 '?a  _instance_hypernym  ?g  ?g  _synset_domain_topic_of  ?b   => ?a  _synset_domain_topic_of  ?b\t0.032092426\t0.395256917\t0.862068966\t100\t253\t116\t?a',
 '?a  _hypernym  ?g  ?g  _synset_domain_topic_of  ?b   => ?a  _synset_domain_topic_of  ?b\t0.178754814\t0.285056295\t0.809593023\t557\t1954\t688\t?a']

In [7]:
dropped_rules_amie_wn18rr[:5]

['?g  _synset_domain_topic_of  ?b  ?a  _verb_group  ?g   => ?a  _synset_domain_topic_of  ?b\t0.012516046\t0.375\t0.75\t39\t104\t52\t?a',
 '?g  _synset_domain_topic_of  ?b  ?g  _verb_group  ?a   => ?a  _synset_domain_topic_of  ?b\t0.012516046\t0.386138614\t0.75\t39\t101\t52\t?a',
 '?g  _hypernym  ?b  ?a  _member_of_domain_region  ?g   => ?a  _member_of_domain_region  ?b\t0.01191766\t0.021072797\t0.733333333\t11\t522\t15\t?b',
 '?a  _has_part  ?g  ?b  _instance_hypernym  ?g   => ?a  _has_part  ?b\t0.029900332\t0.41025641\t0.52173913\t144\t351\t276\t?b',
 '?a  _also_see  ?g  ?b  _also_see  ?g   => ?a  _also_see  ?b\t0.16243264\t0.094831461\t0.115997801\t211\t2225\t1819\t?b']

In [8]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_wn18rr)))/len(dropped_rules_amie_wn18rr))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_wn18rr)))/len(dropped_rules_amie_wn18rr))

PCA: 0.399746019875
Std: 0.19149289562500002


In [9]:
new_rules_amie_wn18rr[:10]

['?a  _hypernym  ?b  ?b  _verb_group  ?a   => ?a  _verb_group  ?b\t0.010460251\t0.909090909\t1\t10\t11\t10\t?b',
 '?b  _hypernym  ?a  ?b  _verb_group  ?a   => ?a  _verb_group  ?b\t0.010460251\t1\t1\t10\t10\t10\t?b']

In [10]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_wn18rr)))/len(new_rules_amie_wn18rr))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_wn18rr)))/len(new_rules_amie_wn18rr))

PCA: 1.0
Std: 0.9545454545000001


In [11]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_wn18rr_1)))/len(same_rules_amie_wn18rr_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_wn18rr_1)))/len(same_rules_amie_wn18rr_1))

print()
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_wn18rr_2)))/len(same_rules_amie_wn18rr_2))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_wn18rr_2)))/len(same_rules_amie_wn18rr_2))

PCA: 0.5133897863076923
Std: 0.2637725748846154

PCA: 0.4715662634615383
Std: 0.21696860796153847


In [12]:
# Before
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_wn18rr)))/len(rules_amie_before_wn18rr))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_wn18rr)))/len(rules_amie_before_wn18rr))

PCA: 0.4866500765588235
Std: 0.24676559152941172


In [13]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_wn18rr)))/len(rules_amie_after_wn18rr))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_wn18rr)))/len(rules_amie_after_wn18rr))

PCA: 0.5093115303571426
Std: 0.26965266842857144


## Mutagenesis

In [14]:
same_rules_amie_mut_1, same_rules_amie_mut_2, rules_amie_before_mut, rules_amie_after_mut, \
new_rules_amie_mut, dropped_rules_amie_mut = compare_rules_amie('AMIE/rules_mutagenesis/rules.txt', 'TransAMIE/rules_mutagenesis/rules_top500.txt')

There were 9 rules before, there are 9 after enrichment

There are 9 same rules, 0 new rules, 0 dropped rules


In [15]:
new_rules_amie_mut[:10]

[]

In [16]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_mut)))/len(rules_amie_before_mut))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_mut)))/len(rules_amie_before_mut))

PCA: 0.5425176136666666
Std: 0.43713459811111105


In [17]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_mut)))/len(rules_amie_after_mut))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_mut)))/len(rules_amie_after_mut))

PCA: 0.5262443997777777
Std: 0.4563609814444444


In [17]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_mut_1)))/len(same_rules_amie_mut_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_mut_1)))/len(same_rules_amie_mut_1))
print()
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_mut_2)))/len(same_rules_amie_mut_2))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_mut_2)))/len(same_rules_amie_mut_2))

PCA: 0.5425176136666666
Std: 0.43713459811111105

PCA: 0.5262443997777777
Std: 0.4563609814444444


In [19]:
# New rules
#print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_mut)))/len(new_rules_amie_mut))
#print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_mut)))/len(new_rules_amie_mut))

## Carcinogenesis

In [20]:
same_rules_amie_car_1, same_rules_amie_car_2, rules_amie_before_car, rules_amie_after_car, \
new_rules_amie_car, dropped_rules_amie_car = compare_rules_amie('AMIE/rules_carcinogenesis/rules.txt', 'TransAMIE/rules_carcinogenesis/rules_top500.txt')

There were 185 rules before, there are 171 after enrichment

There are 147 same rules, 24 new rules, 38 dropped rules


In [21]:
new_rules_amie_car[:10]

['?a  carcinogenesis#salmonella_reduc  ?b   => ?a  carcinogenesis#mouse_lymph  ?b\t0.012903226\t1\t1\t2\t2\t2\t?a',
 '?a  carcinogenesis#chromaberr  ?b  ?a  carcinogenesis#micronuc_f  ?b   => ?a  carcinogenesis#salmonella  ?b\t0.012658228\t0.75\t1\t3\t4\t3\t?a',
 '?a  carcinogenesis#chromaberr  ?b  ?a  carcinogenesis#micronuc_f  ?b   => ?a  carcinogenesis#amesTestPositive  ?b\t0.017167382\t1\t1\t4\t4\t4\t?a',
 '?a  carcinogenesis#amesTestPositive  ?b  ?a  carcinogenesis#drosophila_rt  ?b   => ?a  carcinogenesis#salmonella  ?b\t0.016877637\t0.5\t1\t4\t8\t4\t?a',
 '?a  carcinogenesis#amesTestPositive  ?b  ?a  carcinogenesis#mouse_lymph  ?b   => ?a  carcinogenesis#salmonella  ?b\t0.181434599\t0.704918033\t0.977272727\t43\t61\t44\t?a',
 '?a  carcinogenesis#cytogen_sce  ?b  ?a  carcinogenesis#salmonella  ?b   => ?a  carcinogenesis#amesTestPositive  ?b\t0.253218884\t0.670454545\t0.967213115\t59\t88\t61\t?a',
 '?a  carcinogenesis#drosophila_rt  ?b  ?a  carcinogenesis#drosophila_slrl  ?b   => 

In [22]:
# Before enrichment
print('PCA:', sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_car)))/len(rules_amie_before_car))
print('Std:', sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_car)))/len(rules_amie_before_car))

PCA: 0.7375154245621616
Std: 0.5162657981027027


In [23]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_car)))/len(rules_amie_after_car))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_car)))/len(rules_amie_after_car))

PCA: 0.7109953531228069
Std: 0.4856411325438596


In [24]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_car)))/len(new_rules_amie_car))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_car)))/len(new_rules_amie_car))

PCA: 0.7122427243333332
Std: 0.5198405719583333


In [25]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_car)))/len(dropped_rules_amie_car))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_car)))/len(dropped_rules_amie_car))

PCA: 0.7535989299210527
Std: 0.5282689622105262


In [26]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_car_1)))/len(same_rules_amie_car_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_car_1)))/len(same_rules_amie_car_1))

PCA: 0.7333577837210885
Std: 0.5131629393537415


## FB15k

In [27]:
same_rules_amie_fb_1, same_rules_amie_fb_2, rules_amie_before_fb, rules_amie_after_fb, \
new_rules_amie_fb, dropped_rules_amie_fb = compare_rules_amie('AMIE/rules_fb15k/rules.txt', 'TransAMIE/rules_fb15k/rules_top500.txt')

There were 7591 rules before, there are 5751 after enrichment

There are 5135 same rules, 616 new rules, 2456 dropped rules


In [28]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_fb)))/len(rules_amie_before_fb))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_fb)))/len(rules_amie_before_fb))

PCA: 0.5633267253446179
Std: 0.3071214942907393


In [29]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_fb)))/len(rules_amie_after_fb))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_fb)))/len(rules_amie_after_fb))

PCA: 0.5581248147266537
Std: 0.2979733923030776


In [30]:
# dropped rules, no dropped rules
#sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_fb)))/len(dropped_rules_amie_fb)

In [31]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_fb)))/len(new_rules_amie_fb))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_fb)))/len(new_rules_amie_fb))

PCA: 0.466331240910715
Std: 0.2712862863360386


In [32]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_fb_1)))/len(same_rules_amie_fb_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_fb_1)))/len(same_rules_amie_fb_1))

PCA: 0.5895423653273609
Std: 0.34997305621382757


## YAGO3

In [33]:
same_rules_amie_yago_1, same_rules_amie_yago_2, rules_amie_before_yago, rules_amie_after_yago, \
new_rules_amie_yago, dropped_rules_amie_yago = compare_rules_amie('AMIE/rules_yago3/rules.txt', 'TransAMIE/rules_yago3/rules_top500.txt')

There were 235 rules before, there are 184 after enrichment

There are 179 same rules, 5 new rules, 56 dropped rules


In [34]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_yago)))/len(rules_amie_before_yago))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_yago)))/len(rules_amie_before_yago))

PCA: 0.3242538606127661
Std: 0.19154214194468083


In [35]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_yago)))/len(rules_amie_after_yago))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_yago)))/len(rules_amie_after_yago))

PCA: 0.3089386024945652
Std: 0.15255460118478256


In [36]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_yago)))/len(new_rules_amie_yago))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_yago)))/len(new_rules_amie_yago))

PCA: 0.274665772
Std: 0.21573530720000003


In [37]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_yago)))/len(dropped_rules_amie_yago))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_yago)))/len(dropped_rules_amie_yago))

PCA: 0.26078532192857146
Std: 0.15456280444642861


In [38]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_yago_1)))/len(same_rules_amie_yago_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_yago_1)))/len(same_rules_amie_yago_1))

PCA: 0.34410993975418996
Std: 0.20311109669273736


## Open Bio Link

In [39]:
same_rules_amie_op_1, same_rules_amie_op_2, rules_amie_before_op, rules_amie_after_op, \
new_rules_amie_op, dropped_rules_amie_op = compare_rules_amie('AMIE/rules_openbiolink/rules.txt', 'TransAMIE/rules_openbiolink/rules_top500.txt')

There were 713 rules before, there are 612 after enrichment

There are 575 same rules, 37 new rules, 138 dropped rules


In [40]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_op)))/len(rules_amie_before_op))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_op)))/len(rules_amie_before_op))

PCA: 0.31666675416129036
Std: 0.2804260248106593


In [41]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_op)))/len(rules_amie_after_op))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_op)))/len(rules_amie_after_op))

PCA: 0.27186394783169937
Std: 0.23671334431535968


In [42]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_op)))/len(dropped_rules_amie_op))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_op)))/len(dropped_rules_amie_op))

PCA: 0.2990007797028983
Std: 0.2711057251739131


In [43]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_op)))/len(new_rules_amie_op))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_op)))/len(new_rules_amie_op))

PCA: 0.47948494345945947
Std: 0.42798116121621627


In [44]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_op_1)))/len(same_rules_amie_op_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_op_1)))/len(same_rules_amie_op_1))

PCA: 0.32090658803130456
Std: 0.2826628967234783


## DRKG

In [45]:
same_rules_amie_drkg_1, same_rules_amie_drkg_2, rules_amie_before_drkg, rules_amie_after_drkg, \
new_rules_amie_drkg, dropped_rules_amie_drkg = compare_rules_amie('AMIE/rules_drkg/rules.txt', 'RotAMIE/rules_drkg/rules_top500.txt')

There were 2804 rules before, there are 2747 after enrichment

There are 1310 same rules, 1437 new rules, 1494 dropped rules


In [46]:
for rule in new_rules_amie_drkg:
    if "treats" in rule or "INHIBITOR" in rule:
        print (rule)

?b  DGIDB::INHIBITOR::Gene:Compound  ?a  ?a  INTACT::ASSOCIATION::Compound:Gene  ?b   => ?a  INTACT::PHYSICAL_ASSOCIATION::Compound:Gene  ?b	0.034722222	0.277777778	0.833333333	5	18	6	?b
?a  DRUGBANK::treats::Compound:Disease  ?b  ?a  GNBR::Pa::Compound:Disease  ?b   => ?a  GNBR::Pr::Compound:Disease  ?b	0.015404365	0.571428571	0.631578947	12	21	19	?a
?a  DRUGBANK::treats::Compound:Disease  ?b  ?a  GNBR::C::Compound:Disease  ?b   => ?a  GNBR::Pr::Compound:Disease  ?b	0.016688062	0.590909091	0.619047619	13	22	21	?a
?a  DGIDB::INHIBITOR::Gene:Compound  ?b  ?b  GNBR::N::Compound:Gene  ?a   => ?a  DGIDB::OTHER::Gene:Compound  ?b	0.010481657	0.461538462	0.595744681	84	182	141	?b
?a  DGIDB::INHIBITOR::Gene:Compound  ?b  ?a  DGIDB::OTHER::Gene:Compound  ?b   => ?a  DGIDB::ALLOSTERIC_MODULATOR::Gene:Compound  ?b	0.015873016	0.005997001	0.571428571	4	667	7	?b
?g  DGIDB::INHIBITOR::Gene:Compound  ?b  ?g  INTACT::PHOSPHORYLATION_REACTION::Gene:Gene  ?a   => ?a  DGIDB::ANTIBODY::Gene:Compound  ?b	

In [47]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_drkg)))/len(rules_amie_before_drkg))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_drkg)))/len(rules_amie_before_drkg))

PCA: 0.2894146629104849
Std: 0.16976895299358047


In [48]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_drkg)))/len(rules_amie_after_drkg))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_drkg)))/len(rules_amie_after_drkg))

PCA: 0.2813953425303963
Std: 0.1666508628045142


In [49]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_drkg)))/len(new_rules_amie_drkg))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_drkg)))/len(new_rules_amie_drkg))

PCA: 0.2607996997369521
Std: 0.13668737307446072


In [50]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_drkg)))/len(dropped_rules_amie_drkg))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_drkg)))/len(dropped_rules_amie_drkg))

PCA: 0.2738431037469882
Std: 0.14314852207028117


In [51]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_drkg_1)))/len(same_rules_amie_drkg_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_drkg_1)))/len(same_rules_amie_drkg_1))

PCA: 0.3071733723687026
Std: 0.20012843680992382


In [2]:
#dropped_rules_amie_drkg